# ECOSTRESS data wrangling

From [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears), I've downloaded a whole collection of geotiffs for ECOSTRESS LST for a bounding box around Clear Lake.

Open them all, clip them to the lake shapefile area, compute some zonal stats, plot and save a figure.

In [1]:
import os
import xarray as xr
import rioxarray as rioxr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, mapping

import matplotlib.pyplot as plt
%matplotlib inline

Open the shapefile for Clear lake

In [2]:
# Open the shapefile for Clear Lake
clearlake = gpd.read_file('zip://../data/Clear_Lake_Res.zip')
clearlake.head()

# Set shapefile to same crs as ecostress
clearlake = clearlake.to_crs('epsg:4326')

Set the path to the top level directory where all our ECOSTRESS images are, their file extension, and product name.

In [3]:
eco_tiff_path = '/home/jovyan/data/ECOSTRESS/'
ext = '.tif'
product = 'SDS_LST_doy'

Get a list of all our ECOSTRESS LST files

In [4]:
# get list of all tif files
file_list = []
for root, dirs, files in os.walk(eco_tiff_path):
    for file in files:
        if file.endswith(ext):
            if product in file:
                 file_list.append( os.path.join(root, file) ) 

Open and plot each image

In [16]:
for i, file in enumerate(file_list):
    
    # get the timestring from the filename
    file_datestring = file_list[i].split('/')[-1].split('_')[-2][3:]
    
    # convert to pandas timestamp
    file_timestamp = pd.to_datetime(file_datestring, format='%Y%j%H%M%S')
    
    # Open one of our ECOSTRESS images
    eco_lst = xr.open_rasterio(file)
    
    # Scale the temperature values
    scaled_eco_lst = eco_lst * float(eco_lst.attrs['scale_factor'])
    
    try:
        # Mask using the Clear Lake shapefile
        clearlake_lst = eco_lst.rio.clip(clearlake.geometry.apply(mapping))
    except NoDataInBounds: # if there's no data in the shapefile
        continue
    
    # switch our zero values to nan values
    clearlake_lst = clearlake_lst.where(clearlake_lst > 0)
    
    # scale our clipped dataset and convert from K to C
    scaled_clearlake_lst = ( clearlake_lst * float(eco_lst.attrs['scale_factor']) ) - 273.15
    
    ### Compute zonal statistics
    # flatten our data array
    values = scaled_clearlake_lst.values.flatten()
    # Remove NaN pixel values
    values = values[~np.isnan(values)]
    
    # Only continue on if we have most of the pixels in the scene
    if values.size > 27600:
    
        # compute stats
        clearlake_lst_mean = values.mean()
        clearlake_lst_max = values.max()
        clearlake_lst_min = values.min()
        clearlake_lst_std = values.std()
        
        # Only continue if we probably don't have clouds
        if clearlake_lst_mean > 0:
        
            # make a string for plotting
            summary_stats = 'Mean: {}\nMax: {}\nMin: {}\nStd: {}'.format(np.round(clearlake_lst_mean,1),
                                                                         np.round(clearlake_lst_max,1),
                                                                         np.round(clearlake_lst_min,1),
                                                                         np.round(clearlake_lst_std,1))
            
            ### Plot the clipped ECOSTRESS image and histogram with summary statistics
            fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,3), gridspec_kw={'width_ratios': [3, 1]})
            
            # plot the image clipped to the shapefile
            scaled_clearlake_lst.plot(x='x', y='y', ax=ax1, vmin=-10, vmax=30, cmap='magma')
            clearlake.plot(ax=ax1, edgecolor='blue', facecolor='none');
            ax1.set_aspect(1)
            ax1.set_title('{}'.format(file_timestamp))
        
        
            # plot the histogram
            ax2.hist(values, bins=50, facecolor='k');
            ax2.axvline(x=clearlake_lst_mean, c='r', linestyle='--')
            bbox = dict(boxstyle="round",ec='none',fc='w',alpha=0.5)
            ax2.text(-8,1250,summary_stats, bbox=bbox)
            ax2.set_ylabel('Number of Pixels')
            ax2.set_xlabel('Temperature [C]')
            ax2.set_xlim((-10,50))
            ax2.set_ylim((0,2000))
            ax1.set_title('{}'.format(file_timestamp))
            
            plt.tight_layout()
            
            plt.savefig('../figures/{}.jpg'.format(file_datestring))
            plt.close()

/home/jovyan/data/conda-envs/firewaterenv/lib/python3.6/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/jovyan/data/conda-envs/firewaterenv/lib/python3.6/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/jovyan/data/conda-envs/firewaterenv/lib/python3.6/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the pre

NameError: name 'NoDataInBounds' is not defined